In [ ]:
from aocd import data, models, submit
from io import StringIO
from pathlib import Path
import re

import pandas as pd
import numpy as np

# Load data and examples

In [ ]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [ ]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
todays_examples = todays_puzzle.examples

In [ ]:
data = todays_puzzle.input_data

# Part A

In [ ]:
def fill_right_until_obstacle(grid_data, current_position):
    curr_x, curr_y = current_position
    current_row = grid_data[curr_x]
    n = len(current_row)
    while curr_y < n and current_row[curr_y] != "#":
        current_row[curr_y] = "x"
        curr_y += 1
    curr_y -= 1
    grid_data = np.rot90(grid_data, k=1)
    # need to rotate the current position as well
    curr_x, curr_y = n - 1 - curr_y, curr_x
    return grid_data, (curr_x, curr_y)  # current position

In [ ]:
def part_a(data: str) -> str:
    grid_data = np.array([[x for x in line] for line in data.split()])
    rotation_count = {
        "^": 3,
        "v": 1,
        ">": 0,
        "<": 2,
    }  # to rotate the board anti-clockwise to have the guard going left
    initial_position = np.argwhere(
        (grid_data == "<")
        + (grid_data == ">")
        + (grid_data == "^")
        + (grid_data == "v")
    )
    n = grid_data.shape[0]
    curr_x, curr_y = initial_position[0]
    initial_rotation_count = rotation_count[grid_data[curr_x][curr_y]]

    grid_data[curr_x][curr_y] = "S"
    grid_data = np.rot90(grid_data, k=initial_rotation_count)

    curr_x, curr_y = np.argwhere(grid_data == "S")[0]

    while curr_x != 0:
        grid_data, (curr_x, curr_y) = fill_right_until_obstacle(
            grid_data, (curr_x, curr_y)
        )

    result = (grid_data == "x").sum()
    return str(result)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        print(
            f"Example {example_index} part a: {part_a(example.input_data)} (expected {example.answer_a})"
        )
        assert part_a(str(example.input_data)) == example.answer_a
submit(part_a(data), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [ ]:
# clock-wise rotation
rotation_matrix = np.array([[0, -1], [1, 0]])

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass(frozen=True)
class TurningPointAndHeadings:
    point: tuple
    heading: tuple

In [ ]:
def find_all_turning_points(
    grid_data, initial_position, initial_heading
) -> list | None:
    heading = initial_heading
    n = grid_data.shape[0]

    i, k = initial_position
    turning_points_and_headings = []
    # end condition works only in the examples where the guard
    # does not start at the edge, but for me this works :D
    while i != 0 and i != n - 1 and k != 0 and k != n - 1:
        next_i, next_k = i + heading[0], k + heading[1]
        if grid_data[next_i, next_k] == "#":
            curr_point_and_heading = TurningPointAndHeadings((i, k), tuple(heading))
            if curr_point_and_heading in turning_points_and_headings:
                # Loop detected
                return None
            turning_points_and_headings.append(curr_point_and_heading)
            heading = np.matmul(heading, rotation_matrix)
        else:
            i, k = next_i, next_k
    turning_point = [tph.point for tph in turning_points_and_headings]
    # add exit point
    turning_point.append((i, k))
    return turning_point

In [ ]:
def part_b(data: str) -> str:
    heading_dict = {"^": (-1, 0), "v": (1, 0), "<": (0, -1), ">": (0, 1)}
    grid_data = np.array([[x for x in line] for line in data.split()])
    initial_position = np.argwhere(
        (grid_data == "<")
        + (grid_data == ">")
        + (grid_data == "^")
        + (grid_data == "v")
    )[0]
    n = grid_data.shape[0]
    curr_x, curr_y = initial_position
    initial_heading = heading_dict[grid_data[curr_x][curr_y]]
    turning_points = find_all_turning_points(
        grid_data, initial_position, initial_heading
    )
    sucessfull_obstacle_placements = set()
    result = 0
    for curr_point, next_point in zip(
        [initial_position] + turning_points, turning_points
    ):
        curr_point, next_point = np.array(curr_point), np.array(next_point)
        heading = next_point - curr_point
        distance = max(heading)

        heading_sign = np.sign(heading)
        # normalise heading knowing that we can move in only one direction
        heading = heading_sign * heading.astype(bool).astype(int)
        heading_rot90 = np.matmul(heading, rotation_matrix)

        # We place an obstacle along the path of the guard and check if this works
        obstacle_placement = curr_point
        while True:
            obstacle_placement += heading
            initial_field = grid_data[tuple(obstacle_placement)]
            grid_data[tuple(obstacle_placement)] = "#"
            if (
                find_all_turning_points(grid_data, initial_position, initial_heading)
                is None
            ):
                sucessfull_obstacle_placements.add(tuple(obstacle_placement))
            grid_data[tuple(obstacle_placement)] = initial_field
            if np.all(obstacle_placement == next_point):
                break
    result = len(sucessfull_obstacle_placements)
    return str(result)

In [ ]:
todays_examples[0] = todays_examples[0]._replace(answer_b="6")

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_b != "":
        print(
            f"Example {example_index} part b: {part_b(example.input_data)} (expected {example.answer_b})"
        )
        assert part_b(str(example.input_data)) == example.answer_b
submit(part_b(data), part="b", year=puzzle_year, day=puzzle_day)